If we need to isolate our project but not only take care about the python dependencies and we need to isolated the OS dependencies too, then we have Docker. With Docker you are able to pack all your project in a system that you want and run it in any system machine. For example if you want Ubuntu 20.4 you can have it in a mac or windows machine or other operating systems.

**Why we need Docker?** Could be that we have different projects and each of them need a different version of ubuntu or a specific version of c++ compiler, etc. then we need to isolate the system dependencies. With Docker we can run different docker containers in the same host(computer for example).

**Advantage**: It's easy to put all dependencies needed for a project in a docker container and deploy to the cloud.

- **Install Docker**:

```bash
sudo apt update
sudo apt install docker.io
```
- **Add current user to docker group**

The next comand is to run docker without ```sudo```.

```bash
sudo usermod -aG docker $USER
logout
```

Then logback to the VM.


## Run a Docker Image

```bash
docker run -it --rm --entrypoint=bash
```



## Build a Docker Image

se necesita una imagen de docker como base para construir otras imagenes de docker
In Docker image file there are settings and dependecies we have in our project. To find Docker images that you need you can simply search the [Docker](https://hub.docker.com/search?q=&type=image) website.

Here a Docker file is written we'll explain it below.(There should be no comments in Docker file, So remove the comments if you want to copy it)

```Dockerfile
FROM python:3.8.12-slim                  # First install the python 3.8, the slim version have less size
RUN pip install pipenv                   # Install pipenv library in Docker 
WORKDIR /app                             # Use '/app' as working directory(create if it doesn't exist) 
COPY ["Pipfile", "Pipfile.lock", "./"]   # Copy the Pip files into our working derectory 
RUN pipenv install --deploy --system     # Install dependecies from Pipfile.lock without creating a virtual environment 
COPY ["*.py", "churn-model.bin", "./"]   # Copy any python files and the model we had to the working directory 
EXPOSE 9696                              # Expose the 9696 port to communicate with Docker outside it
ENTRYPOINT ["gunicorn", "--bind", "0.0.0.0:9696", "churn_serving:app"]      # If we run the Docker image, we want our churn app to be running.

```

## Docker 

`Docker is an open platform for developing, shipping, and running applications. Docker enables you to separate your applications from your infrastructure so you can deliver software quickly. With Docker, you can manage your infrastructure in the same ways you manage your applications. By taking advantage of Docker’s methodologies for shipping, testing, and deploying code quickly, you can significantly reduce the delay between writing code and running it in production.`

### The Docker platform

Docker provides the ability to package and run an application in a loosely isolated environment called a container. The isolation and security allow you to run many containers simultaneously on a given host. Containers are lightweight and contain everything needed to run the application, so you do not need to rely on what is currently installed on the host. You can easily share containers while you work, and be sure that everyone you share with gets the same container that works in the same way.

Docker provides tooling and a platform to manage the lifecycle of your containers:

* Develop your application and its supporting components using containers.
* The container becomes the unit for distributing and testing your application.
* When you’re ready, deploy your application into your production environment, as a container or an orchestrated service. This works the same whether your production environment is a local data center, a cloud provider, or a hybrid of the two.


### What can I use Docker for?
* #### Fast, consistent delivery of your applications
* #### Responsive deployment and scaling
* #### Running more workloads on the same hardware

### Docker architecture

![Dcoker Architecture](https://docs.docker.com/engine/images/architecture.svg)


#### Dockerfile

* A Dockerfile is a set of instructions used to create a Docker image. Each instruction is an operation used to package the application, such as installing dependencies, compile the code, or impersonate a specific user.
* A Docker image is composed of multiple layers, and each layer is represented by an instruction in the Dockerfile. All layers are cached and if an instruction is modified, then during the build process only the changed layer will be rebuild. As a result, building a Docker image using a Dockerfile is a lightweight and quick process.
* To construct a Dockerfile, it is necessary to use the pre-defined instructions, such as:

```Dockerfile
FROM           - to set the base image
RUN            - to execute a command
COPY & ADD     - to copy files from host to the container
CMD            - to set the default command to execute when the container starts
EXPOSE         - to expose an application port 

```

## Example : 

```Dockerfile
# Base image:
FROM agrigorev/zoomcamp-model:3.8.12-slim                

LABEL maintainer="Name of the Maintainer"

ENV PYTHONUNBUFFERED=TRUE

# -- Install Pipenv:
RUN pip --no-cache-dir install pipenv

# -- Install Application into container:
RUN set -ex && mkdir /app
WORKDIR /app

# COPY Pipfile.lock Pipfile.lock
COPY ["Pipfile", "Pipfile.lock", "./"]

# -- Install dependencies:
RUN set -ex && pipenv install --deploy --system

# Copy files from host to the container
COPY ["*.py", "churn-model1.bin","churn-dv.bin", "./"]

# Port where the App will be Exposed
EXPOSE 9696

ENTRYPOINT ["gunicorn", "--bind", "0.0.0.0:9696", "churn_flask_serving:app"] 
```


#### Dockerimage

* Once a Dockerfile is constructed, these instructions are used to build a Docker image.
* A Docker image is a read-only template that enables the creation of a runnable instance of an application.
* In a nutshell, a Docker image provides the execution environment for an application, including any essential code, config files, and dependencies.
* A Docker image can be built from an existing Dockerfile using the docker build command. Below is the syntax for this command:

```bash
# build an image
# OPTIONS - optional;  define extra configuration
# PATH - required;  sets the location of the Dockefile and  any referenced files 

docker build [OPTIONS] PATH

# Where OPTIONS can be:
-t, --tag            - set the name and tag of the image
-f, --file           - set the name of the Dockerfile
--build-arg          - set build-time variables

# Find all valid options for this command 
docker build --help

```


## Example 

* Build the image
```bash
docker build -t churn-prediction .
```

* Run it 
```bash
docker run -it -p 9696:9696 churn-prediction:latest
```

#### Docker Registry
* The last step in packaging an application using Docker is to store and distribute it. So far, we have built and tested an image on the local machine, which does not ensure that other engineers have access to it. As a result, the image needs to be pushed to a public Docker image registry, such as DockerHub, Harbor, Google Container Registry, and many more. 
* To tag an existing image on the local machine, the docker tag command is available. Below is the syntax for this command:

```bash
# tag an image
# SOURCE_IMAGE[:TAG]  - required and the tag is optional; define the name of an image on the current machine 
# TARGET_IMAGE[:TAG] -  required and the tag is optional; define the repository, name, and version of an image
docker tag SOURCE_IMAGE[:TAG] TARGET_IMAGE[:TAG]
```

* Once the image is tagged, the final step is to push the image to a registry. For this purpose, the docker push command can be used. Below is the syntax for this command:
```bash
# push an image to a registry 
# NAME[:TAG] - required and the tag is optional; name, set the image name to be pushed to the registry
docker push NAME[:TAG]
```

* `OPTIONS`          - define extra configuration through flags
* `IMAGE`            - sets the name of the image
* `NAME`             - set the name of the image
* `COMMAND and ARG`  - instruct the container to run specific commands associated with a set of arguments

# Example : 
* tag it 
```bash
docker tag churn-prediction ayoubberd/churn-prediction:latest
```

* Push it
```bash
docker push ayoubberd/churn-prediction:latest
```



# The image can be pulled from 

```bash
docker pull ayoubberd/churn-prediction:latest
```

In [ ]:
# Separarlo por secciones